## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Краткое вступление в работу

В данной работе необходимо ответить на вопросы:
- 1) Есть ли зависимость между наличием детей и возвратом кредита в срок?
- 2) Есть ли зависимость между семейным положением и возвратом кредита в срок?
- 3) Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
- 4) Как разные цели кредита влияют на его возврат в срок?

Прежде чем ответить на вопросы, необходимо провести предварительную работу с данными: 
Открыть таблицу и изучить общую информацию о данных, определить и заполнить пропущенные значения,заменить вещественный тип данных на целочисленный,удалить дубликаты, провести лемматизацию в значениях столбца с целями получения кредита,категоризировать данные: выделить «словари» для этого набора данных.

Описание данных: 
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd #выполнили импорт 
from nltk.stem import SnowballStemmer 
from pymystem3 import Mystem
from collections import Counter
credit = pd.read_csv('/datasets/data.csv') # считали данные по указанному пути.

In [2]:
credit.info() # а вот и данные нашей таблицы, вся необходимая информация о типах данных, количестве строк, 
                 #по которым мы можем сделать необходимые выводы.
credit.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Мы видим в таблице  12 столбцов и количество в каждом из них. С типом данных такими как float, int и object.

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
credit.isnull().sum()
credit['days_employed']=credit['days_employed'].apply(abs)#убираем отрицательные значения в столбце 
days_employed_mean = credit['days_employed'].mean()
credit['days_employed'] = credit['days_employed'].fillna(value = days_employed_mean)
total_income_mean = credit['total_income'].mean()
credit['total_income'] = credit['total_income'].fillna(value = total_income_mean)
credit.info()
credit.head(20)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

Методом isnull() выяснили,что пропущенные значения имеются в двух столбцах: 'days_employed'= 19351 ,'total_income' = 19351. Значения в столбцах количественные, поэтому мы можем пропуски в таких переменных заменить характерными значениями, для этого годится вычисления медианы через метод .mean(),затем  для заполнения пропусков мы используем метод fillna(), так как метод fillna() подходит для работы с пропусками типа NaN, который отвечает за отсутствующее в ячейке число.  
Причины для  возникновения пропусков: возможно клиенты банка предоставляют не весь пакет документов и справок. 


### Замена типа данных

In [4]:
credit['days_employed'] = credit['days_employed'].astype('int')
credit['total_income'] = credit['total_income'].astype('int')
credit.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

  В двух столбцах days_employed, total_income тип данных был вещественный (float), так как метод to_numeric() переводит все числа в float, то на помощь пришел метод astype(), который как раз таки переводит числа из float в int в нашем случае.      

### Обработка дубликатов

In [5]:
credit.duplicated().sum() #подсчитали количество дубликатов

54

In [6]:
credit['education'] = credit['education'].str.lower() #привели к нижнему регистру в столбце education
credit = credit.drop_duplicates().reset_index(drop=True)# удалили дубликаты в таблице, сохранив в датафрейм без дубликатов с сохранением индексов 
credit.duplicated().sum() #подсчитали значение дубликатов в таблице 

0

In [7]:
credit.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null int64
purpose             21454 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

В таблице  имеются дубликаты, выяснили мы это с помощью команды duplicated().sum(). С помощью метода drop_duplicates() по таблице удалили дубликаты   и подсчитали наличие на дубликаты методом duplicated.sum() и получили цифру 0, а значит мы упешно применили используемые методы. 
Так же в столбце education привели все значения к нижнему регистру с помощью метода str.lower(), убрав неверные данные.  

### Лемматизация

In [8]:
m = Mystem()
words_map = {} #создан словарь
lemmas = m.lemmatize(credit['purpose'][0])
for row in credit['purpose']: 
    #print(m.lemmatize(row))
    for word in m.lemmatize(row): 
        if word not in words_map.keys():  #цикл в котором добавляются в словарь все леммы из коллекции и их количество 
            words_map[word] = 1 
        else:
            words_map[word] += 1   
print(words_map) 

{'покупка': 5897, ' ': 33570, 'жилье': 4460, '\n': 21454, 'приобретение': 461, 'автомобиль': 4306, 'дополнительный': 906, 'образование': 4013, 'сыграть': 765, 'свадьба': 2324, 'операция': 2604, 'с': 2918, 'на': 2222, 'проведение': 768, 'для': 1289, 'семья': 638, 'недвижимость': 6351, 'коммерческий': 1311, 'жилой': 1230, 'строительство': 1878, 'собственный': 635, 'подержать': 478, 'свой': 2230, 'со': 627, 'заниматься': 904, 'сделка': 941, 'подержанный': 486, 'получение': 1314, 'высокий': 1374, 'профильный': 436, 'сдача': 651, 'ремонт': 607}


In [9]:
def lemma_purpose(row): 
    lemma_purposes = m.lemmatize(row['purpose']) 
    if 'жилье' in lemma_purposes:
        return 'жилье'
    if 'свадьба' in lemma_purposes:
        return 'свадьба'
    if 'образование' in lemma_purposes:
        return 'образование'
    if 'автомобиль' in lemma_purposes: 
        return 'автомобиль'
    if 'ремонт' in lemma_purposes: 
        return 'ремонт'
    if 'недвижимость' in lemma_purposes: 
        return 'недвижимость'
    
    return 'что_то_другое'
 
credit['purpose_classified'] = credit.apply(lemma_purpose, axis=1)# затем записываем в новый столбец полученные условия и используем метод apply(),
                                                                  #где аргументом является наша созданная функция
credit.head(10)#выводим

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_classified
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,жилье
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,жилье
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,жилье
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,жилье
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,жилье


In [10]:
credit['purpose_classified'].value_counts()


недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_classified, dtype: int64

### Вывод

Для того, чтобы провести анализ наиболее часто встречающихся целей кредита я использовала метод 
лемматизации, благодаря которому строковые значения приводятся к первоначальной словарной форме тем самым, избавляя нас от дублей и искажений результатов данных. 
Чтобы не проводить анализ вручную я попыталась автоматизировать данный процесс и для этого был создан цикл, соответственно в нем создаётся словарь, в который добавляются все леммы из коллекции и их количество, таким образом мы определим самые часто встречаемые слова. Словарь с подсчитанными значениями был создан, но не в читаемом ввиде и с ненужными значениями для того,чтобы отфильтровать нужные значения строкового типа была создана функция с методом лемматизации, которая пробегала по всему столбцу и подсчитывала необходимые значения, которые были заданы в условиях функции.
Таким образом мы провели необходимые расчеты для того, чтобы  в дальнейшей работе использовать эти данные для нахождения закономерности, взаимосвязи между целями кредита и его возвратом.

### Категоризация данных

In [11]:
credit['total_income'].quantile([0.25, 0.5, 0.75, 1.0])#использую функцию квантиля,разделив значения на 25-ый(процентиль), 50(та же самая медиана) и 75-ый.


0.25     107623.00
0.50     156043.50
0.75     195813.25
1.00    2265604.00
Name: total_income, dtype: float64

In [12]:

def income(total_income): #создаем функцию 
    if total_income < 107623: #условие при котором отсеивается доход больше 107623
        return 'низкий доход'# возвращаем значения, если данные подходят под эту категорию 
    if total_income < 156043.50: #условие при котором отсеивается доход больше 156043.50
        return 'средний доход'#условие при котором отсеивается доход больше или равно 195813.25
    if total_income < 195813.25: 
        return 'выше среднего'
    if total_income <= 2265604: 
        return 'высокий доход'
    
credit['income'] = credit['total_income'].apply(income)
credit['income'].value_counts()
credit_pivot2 = credit.pivot_table(index = 'income',  columns = 'debt', values  = 'days_employed',   aggfunc = 'count')
credit_pivot2.columns = ['Нет долга', 'Есть долг']
credit_pivot2.head()

,Нет долга,Есть долг
income,,
высокий доход,4981,383
выше среднего,4900,463
низкий доход,4937,427
средний доход,4895,468


### Вывод

В категоризации данных мы привели к читаемому виду наши данные,применяя сводные таблицы.На глаз можем прикинуть в какой из категории больше долгов. Выделила один словарь: 
- В словаре использовали столбцы с количеством доходов клиентов и так же наличием долгов. Для того, чтобы проанализировать зависимость между столбцами нужна классификация по размеру зарплаты. Для этого мы использовали функцию quantile([0.25, 0.5, 0.75]) с разбитыми промежутками. Затем написали функцию   которая причисляет к определенной группе, благодаря условию, которое мы задали. Затем результат оформили в сводной таблице для наглядных результатов.   

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [13]:
credit['children']=credit['children'].apply(abs)#убираем отрицательные значения

In [14]:
credit['children'].value_counts() #подсчитываем уникальные значения

0     14091
1      4855
2      2052
3       330
20       76
4        41
5         9
Name: children, dtype: int64

In [15]:
credit_pivot = credit.pivot_table(index = 'children',  columns = 'debt', values  = 'days_employed',   aggfunc = 'count')#создаем сводную таблицу
credit_pivot.columns = ['Нет долга', 'Есть долг']#название столбцов 
credit_pivot = credit_pivot.dropna().reset_index(drop=True)#удалили строку, там где имеются пропущенные значения. Так как нам необходимо увидеть лишь закономерность между наличием  и отсутствием детей, а не количеством 
credit_pivot.head(10)#построили сводную таблицу 

,Нет долга,Есть долг
0,13028.0,1063.0
1,4410.0,445.0
2,1858.0,194.0
3,303.0,27.0
4,37.0,4.0
5,68.0,8.0


In [16]:
credit_pivot['total_debt'] = credit_pivot['Есть долг']+ credit_pivot['Нет долга']
credit_pivot.loc[:,'total_debt']

0    14091.0
1     4855.0
2     2052.0
3      330.0
4       41.0
5       76.0
Name: total_debt, dtype: float64

In [17]:
credit_pivot['total_attitude'] = credit_pivot['Есть долг']/credit_pivot['total_debt']
credit_pivot

,Нет долга,Есть долг,total_debt,total_attitude
0,13028.0,1063.0,14091.0,0.075438
1,4410.0,445.0,4855.0,0.091658
2,1858.0,194.0,2052.0,0.094542
3,303.0,27.0,330.0,0.081818
4,37.0,4.0,41.0,0.097561
5,68.0,8.0,76.0,0.105263


In [18]:
credit_pivot['total_attitude'].sort_values(ascending=True)

0    0.075438
3    0.081818
1    0.091658
2    0.094542
4    0.097561
5    0.105263
Name: total_attitude, dtype: float64

### Вывод

Для того, чтобы ответить на данный вопрос,мы использовали сводную таблицу для наглядных данных, затем провели арифметические действия. Судя по отношению, которое мы получили  видим, что чем больше детей, тем больше долгов у клиентов банка, соответственно чем выше это значение, тем чаще люди той или иной категории не возвращают кредит.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [19]:
credit['family_status'].value_counts()

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

In [20]:
credit_pivot1 = credit.pivot_table(index = 'family_status',  columns = 'debt', values  = 'days_employed',   aggfunc = 'count')
credit_pivot1.columns = ['Нет долга', 'Есть долг']
credit_pivot1.head(10)

,Нет долга,Есть долг
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3763,388
женат / замужем,11408,931


In [21]:
credit_pivot1['total_debt1'] = credit_pivot1['Есть долг'] + credit_pivot1['Нет долга']
credit_pivot1.loc[:,'total_debt1']

family_status
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
гражданский брак          4151
женат / замужем          12339
Name: total_debt1, dtype: int64

In [22]:
credit_pivot1['total_attitude1'] = credit_pivot1['Есть долг']/credit_pivot1['total_debt1']
credit_pivot1.head()

,Нет долга,Есть долг,total_debt1,total_attitude1
family_status,,,,
Не женат / не замужем,2536,274,2810,0.097509
в разводе,1110,85,1195,0.071130
вдовец / вдова,896,63,959,0.065693
гражданский брак,3763,388,4151,0.093471
женат / замужем,11408,931,12339,0.075452


In [23]:
credit_pivot1['total_attitude1'].sort_values(ascending=False)

family_status
Не женат / не замужем    0.097509
гражданский брак         0.093471
женат / замужем          0.075452
в разводе                0.071130
вдовец / вдова           0.065693
Name: total_attitude1, dtype: float64

### Вывод

В данном исследовании мы руководствовались такой же логике как и при ответе в первом вопросе. 
Судя по результатам, которые мы получили меньше долгов имеют женатые пары,  вдовец / вдова, люди в разводе. Одинокие люди, либо люди состоящие в гражданском браке имеют болье всего долгов. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
credit_pivot2 = credit.pivot_table(index = 'income',  columns = 'debt', values  = 'days_employed',   aggfunc = 'count')
credit_pivot2.columns = ['Нет долга', 'Есть долг']
credit_pivot2.head()

,Нет долга,Есть долг
income,,
высокий доход,4981,383
выше среднего,4900,463
низкий доход,4937,427
средний доход,4895,468


In [25]:
credit_pivot2['total_debt2'] = credit_pivot2['Есть долг']+ credit_pivot2['Нет долга']
credit_pivot2.loc[:,'total_debt2']

income
высокий доход    5364
выше среднего    5363
низкий доход     5364
средний доход    5363
Name: total_debt2, dtype: int64

In [26]:
credit_pivot2['total_attitude2'] = credit_pivot2['Есть долг']/credit_pivot2['total_debt2']

credit_pivot2['total_attitude2'].sort_values(ascending=False)

income
средний доход    0.087265
выше среднего    0.086332
низкий доход     0.079605
высокий доход    0.071402
Name: total_attitude2, dtype: float64

### Вывод

По результатам нашего исследования мы получили, что люди со средним достатком или доходом выше среднего имеют больше долгов, нежели люди с меньшим и высоким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [27]:
credit['purpose_classified'].value_counts()
credit_pivot3 = credit.pivot_table(index = 'purpose_classified',  columns = 'debt', values  = 'days_employed',   aggfunc = 'count')
credit_pivot3.columns = ['Нет долга', 'Есть долг']
credit_pivot3.head()

,Нет долга,Есть долг
purpose_classified,,
автомобиль,3903,403
жилье,4152,308
недвижимость,5877,474
образование,3643,370
свадьба,2138,186


In [28]:
credit_pivot3['total_debt'] = credit_pivot3['Есть долг'] + credit_pivot3['Нет долга']
credit_pivot3.loc[:,'total_debt']

purpose_classified
автомобиль      4306
жилье           4460
недвижимость    6351
образование     4013
свадьба         2324
Name: total_debt, dtype: int64

In [29]:
credit_pivot3['total_attitude'] = credit_pivot3['Есть долг']/credit_pivot3['total_debt']
credit_pivot3.head()

,Нет долга,Есть долг,total_debt,total_attitude
purpose_classified,,,,
автомобиль,3903,403,4306,0.093590
жилье,4152,308,4460,0.069058
недвижимость,5877,474,6351,0.074634
образование,3643,370,4013,0.092200
свадьба,2138,186,2324,0.080034


In [30]:
credit_pivot3['total_attitude'].sort_values(ascending=False)

purpose_classified
автомобиль      0.093590
образование     0.092200
свадьба         0.080034
недвижимость    0.074634
жилье           0.069058
Name: total_attitude, dtype: float64

### Вывод

По итоговому столбцу в нашей таблице меньше долгов имеют люди, которые берут в кредит жилье и на свадьбу. Больший процент по долгам имеют клиенты, взявший кредит на автомобиль и образование. 

### Шаг 4. Общий вывод

В данной работе были выявлены: 
1. Пропуски в столбцах. Причины для  возникновения пропусков: возможно клиенты банка предоставляют не весь пакет документов и справок.
2. Дубликаты в таблице. Они искажают данные, по этому были удалены из таблицы.
3. Так же с помощью лемматизации были подсчитаны и  выявлены самые распространненые цели по кредитам. 
   По результатам нашего исследования мы выявили некоторые закономерности, то есть были найдены ответы на вопросы: 
-  Есть ли зависимость между наличием детей и возвратом кредита в срок?
      Чем больше количество детей, тем выше процент не вернувших долг, соответственно чем выше это значение, тем чаще люди той или иной категории не возвращают кредит.
-  Есть ли зависимость между семейным положением и возвратом кредита в срок?
      Так же это влияет на семейное положение, клиенты состоящие в браке имеют меньший процент по долгам, а свободные люди напротив.
-  Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
      По результатам нашего исследования мы получили, что люди со средним достатком или доходом выше среднего имеют больше долгов, нежели люди с меньшим и высоким доходом.
-  Как разные цели кредита влияют на его возврат в срок?
      По итоговому столбцу в нашей таблице меньше долгов имеют люди, которые берут в кредит жилье и на свадьбу. Больший процент   по   долгам имеют клиенты, взявший кредит на автомобиль и образование.
      